## BA4G

**Implement LeaderboardCyclopeptideSequencing**

Given: An integer N and a collection of integers Spectrum.

Return: LeaderPeptide after running LeaderboardCyclopeptideSequencing(Spectrum, N).

Link: https://rosalind.info/problems/ba4g/

In [1]:
def get_mass():
    mass = {
        "G": 57,
        "A": 71,
        "S": 87,
        "P": 97,
        "V": 99,
        "T": 101,
        "C": 103,
        "I": 113,
        "L": 113,
        "N": 114,
        "D": 115,
        "K": 128,
        "Q": 128,
        "E": 129,
        "M": 131,
        "H": 137,
        "F": 147,
        "R": 156,
        "Y": 163,
        "W": 186,
    }

    return mass

Funkcije iz prethodnog zadatka koja se koristi

In [2]:
import numpy as np

In [3]:
# small changes - razlika je jer ovdje primamo brojeve a tamo smo primali stringove
def cyclospectrum(peptide):

    n = len(peptide)
    mass = get_mass()
    cyclospectrum = []

    cycle = peptide + peptide[:-1]
    
    cyclospectrum.append(0)
    
    # razlicito u odnosu na BA4C
    cyclospectrum.append(sum(peptide))

    for i in range(n):
      for j in range(1, n):
        subpeptide = cycle[i : i + j]
        # razlicito u odnosu na BA4C
        cyclospectrum.append(sum(subpeptide))

    return sorted( cyclospectrum)

In [4]:
def CyclicPeptideScoring(peptide,experimental_spectrum):

  theoretical_spectrum = cyclospectrum(peptide)
  score = 0
  ex = experimental_spectrum.copy()

  for item in theoretical_spectrum:
    if item in ex:
      score += 1
      ex.remove(item)

  return score

In [5]:
def LinearSpectrum(peptide):

  prefix_mass  = [0]
  linear_spectrum = [0]
 
  for i in range(len(peptide)):
    # promjena u odnosu na BA4J jer su tamo peptidi bili predstavljeni slovima 
    current = prefix_mass[i] +  peptide[i]
    prefix_mass.append(current)

  for i in range(len(peptide)):
    for j in range(i + 1, len(peptide) + 1):
      linear_spectrum.append(prefix_mass[j] - prefix_mass[i])
  
  return sorted(linear_spectrum) 

In [6]:
def LinearScoring(peptide, experimental_spectrum):

  theoretical_spectrum = LinearSpectrum(peptide)

  score = 0

  for item in experimental_spectrum:
    if item in theoretical_spectrum:
      score += 1
      theoretical_spectrum.remove(item)

  return score

In [7]:
def Trim(leaderboard, linear_spectrum, N):

  linear_scores = {}
  leaderboard_new = []

  for i,peptide in enumerate(leaderboard):
    linear_scores[i] = LinearScoring(peptide,linear_spectrum)
 
  linear_scores_list = sorted(linear_scores.values(), reverse = True)
  
  if len(leaderboard) <= N:
   return leaderboard

  for index, score in linear_scores.items():
     if score >= linear_scores_list[N-1]:
       leaderboard_new.append(leaderboard[index])


  return leaderboard_new

In [8]:
def Output_format(peptide):
  p = ''
  output = []
  for pep in peptide:
    p += str(pep) + '-'
  # :-1 da se makne - sa kraja, inace bi bilo 113-128-186-
  print(p[:-1])

Novo

In [9]:
def Is_Consistent(peptide, spectrum):

  for mass in LinearSpectrum(peptide):
    if not mass in spectrum:
      return False
      
  return True

In [10]:
# doodan samo is consistent u odnosu na BA4E
def Expand(peptides,spectrum):
  
  mass = get_mass()
  mass_list = np.unique(list(mass.values()))
  new_list = []

  for peptide in peptides:
    for mass in mass_list:
      x = peptide.copy()
      x.append(mass)
      if Is_Consistent(x,spectrum):
        new_list.append(x)

  return new_list

In [11]:
def LeaderboardCyclopeptideSequencing(N,spectrum):
 
    leaderboard = [[]]
    leaderPeptide = []

    while leaderboard:
      
      leaderboard  = Expand(leaderboard,spectrum)
      parent_mass = spectrum[-1]

      for peptide in leaderboard:
        if sum(peptide) == parent_mass:
          if CyclicPeptideScoring(peptide, spectrum) > CyclicPeptideScoring(leaderPeptide, spectrum):
            leaderPeptide = peptide
        elif sum(peptide) > parent_mass:
          leaderboard.remove(peptide)

      leaderboard  = Trim(leaderboard, spectrum, N)

    return leaderPeptide

In [12]:
# sample dataset
spectrum = [0, 71, 113, 129, 147 ,200, 218, 260 ,313,331 ,347 ,389 ,460]

In [13]:
LeaderboardCyclopeptideSequencing(10,spectrum)

[113, 147, 71, 129]

In [14]:
# dataset
with open('/content/rosalind_ba4g (5).txt') as file:
  N = file.readline().strip()
  spectrum = [int(s) for s in file.readline().strip().split()]

In [15]:
result = LeaderboardCyclopeptideSequencing(int(N),spectrum)

In [17]:
Output_format(result)

71-101-113-71-128-186-114-129-131-71-103-163-101
